In [23]:
import h5py
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import Legend, LegendItem

import aframe_plots.plot_utils as utils
from aframe_plots.config import DataPaths

output_notebook()


def get_err(V, dV):
    denom = np.sqrt(4 * np.pi) * 3 * V
    denom[V == 0] = 1
    dV[V == 0] = 0
    return dV / denom**(2/3)


def load_results(eval):
    result = {}
    with h5py.File(eval, "r") as f:
        far = f["far"][:]
        far *= 3600 * 24 * 30
        idx = np.where(far < 1000)
        result["far"] = far[idx]
        result["sensitive-distance"] = f["sensitive-distance"][idx]

        volume = f["sensitive-volume"][idx]
        err = f["sensitive-volume-error"][idx]
        result["err"] = get_err(volume, err)
    return result


comp_results = {}
for pipeline in DataPaths.MDC_RESULTS.iterdir():
    ds4 = pipeline / "ds4"
    if not ds4.exists():
        continue

    result = load_results(ds4 / "eval.hdf")
    comp_results[pipeline.name] = result

pipelines = sorted(comp_results)
comp_results["aframe"] = load_results(DataPaths.AFRAME_MDC_RESULTS)
pipelines.insert(0, "aframe")

Loading BokehJS ...

In [33]:
p = utils.get_figure(
    x_axis_type="log",
    x_axis_label=r"$$\text{False Alarm Rate [months}^{-1}\text{]}$$",
    y_axis_label=r"$$\text{Sensitive Distance [Mpc]}$$",
    width=800
)

items = []
for i, pipeline in enumerate(pipelines):
    color = utils.palette[i]
    results = comp_results[pipeline]

    r = p.line(
        results["far"],
        results["sensitive-distance"],
        line_color=color,
        line_width=0.8
    )
    item = LegendItem(renderers=[r], label=pipeline.replace("_", " "))
    items.append(item)

    utils.plot_err_bands(
        p,
        results["far"][::-1],
        results["sensitive-distance"][::-1],
        results["err"],
        fill_color=color,
        fill_alpha=0.5,
        line_width=0.2,
        line_color=color
    )

legend = Legend(items=items)
p.add_layout(legend, "right")
show(p)